In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, accuracy_score
import os

# Configurar para usar GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Usar la primera GPU disponible
import tensorflow as tf

# Verificar si la GPU está disponible y configurarla
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Configurar memoria GPU para crecimiento según necesidad
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU disponible: {gpus}")
    except RuntimeError as e:
        print(f"Error al configurar GPU: {e}")
else:
    print("No se detectó GPU. Usando CPU.")

# Verificar dispositivos disponibles
from tensorflow.python.client import device_lib
print("Dispositivos disponibles:")
print(device_lib.list_local_devices())

# Importar los modelos desde tu módulo
from convolutional import ConvRegressor, ConvClassifier, run_kfold_cv

# Cargar los datos
print("\nCargando datos...")
df = pd.read_csv("data/datos_variables_seleccionadas.csv")
print(f"Dimensiones del DataFrame: {df.shape}")
print(f"Columnas: {df.columns.tolist()}")


No se detectó GPU. Usando CPU.
Dispositivos disponibles:
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10447724029578519845
xla_global_id: -1
]

Cargando datos...
Dimensiones del DataFrame: (565658, 29)
Columnas: ['periodo', 'estu_tipodocumento', 'cole_area_ubicacion', 'cole_caracter', 'cole_cod_dane_establecimiento', 'cole_cod_depto_ubicacion', 'cole_codigo_icfes', 'cole_depto_ubicacion', 'cole_jornada', 'cole_mcpio_ubicacion', 'cole_naturaleza', 'estu_depto_presentacion', 'estu_depto_reside', 'estu_genero', 'estu_mcpio_presentacion', 'estu_mcpio_reside', 'fami_cuartoshogar', 'fami_educacionmadre', 'fami_educacionpadre', 'fami_estratovivienda', 'fami_tieneautomovil', 'fami_tienelavadora', 'desemp_ingles', 'punt_ingles', 'punt_matematicas', 'eco', 'fami_cuartoshogar_int', 'edad', 'fami_nivel_tecnologia']


In [ ]:
print("\nDividiendo datos en train y test (80/20)...")
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
print(f"Train: {df_train.shape}, Test: {df_test.shape}")

# 1. Prueba del modelo de regresión con 5 épocas
print("\n=== PRUEBA MODELO DE REGRESIÓN (punt_matematicas) ===")
target_col_reg = 'punt_matematicas'

# Crear y entrenar el modelo
print("Inicializando modelo de regresión...")
reg_model = ConvRegressor(
    embedding_dim=4,
    conv_filters=[16, 32],
    dense_units=64
)

print("Construyendo modelo...")
reg_model.build_model(df_train.copy(), target_col_reg)

print("Entrenando modelo (5 épocas)...")
reg_model.train(target_col=target_col_reg, epochs=5, batch_size=32)

# Evaluar en el conjunto de test
print("Evaluando en conjunto de test...")
y_true_reg = df_test[target_col_reg].values
y_pred_reg = reg_model.predict(df_test.drop(columns=[target_col_reg, 'eco'])).squeeze()
mae = mean_absolute_error(y_true_reg, y_pred_reg)
print(f"MAE en test: {mae:.4f}")

# 2. Prueba del modelo de clasificación con 5 épocas
print("\n=== PRUEBA MODELO DE CLASIFICACIÓN (eco) ===")
target_col_clf = 'eco'

# Crear y entrenar el modelo
print("Inicializando modelo de clasificación...")
clf_model = ConvClassifier(
    embedding_dim=4,
    conv_filters=[16, 32],
    dense_units=64
)

print("Construyendo modelo...")
clf_model.build_model(df_train.copy(), target_col_clf)

print("Entrenando modelo (5 épocas)...")
clf_model.train(target_col=target_col_clf, epochs=5, batch_size=32)

# Evaluar en el conjunto de test
print("Evaluando en conjunto de test...")
y_true_clf = df_test[target_col_clf].values
y_pred_clf = clf_model.predict(df_test.drop(columns=[target_col_reg, target_col_clf])).squeeze()
y_pred_clf_binary = (y_pred_clf > 0.5).astype(int)
accuracy = accuracy_score(y_true_clf, y_pred_clf_binary)
print(f"Accuracy en test: {accuracy:.4f}")

# 3. Prueba de validación cruzada con 3 épocas
print("\n=== PRUEBA DE VALIDACIÓN CRUZADA (3 épocas) ===")
print("Modelo de Regresión:")
run_kfold_cv(ConvRegressor, df, target_col_reg, task_type='regression', k=5, epochs=3)

print("\nModelo de Clasificación:")
run_kfold_cv(ConvClassifier, df, target_col_clf, task_type='classification', k=5, epochs=3)

# 4. Prueba de validación cruzada con 5 épocas
print("\n=== PRUEBA DE VALIDACIÓN CRUZADA (5 épocas) ===")
print("Modelo de Regresión:")
run_kfold_cv(ConvRegressor, df, target_col_reg, task_type='regression', k=5, epochs=5)

print("\nModelo de Clasificación:")
run_kfold_cv(ConvClassifier, df, target_col_clf, task_type='classification', k=5, epochs=5)

print("\n=== PRUEBAS COMPLETADAS ===")